**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

 # Cross Linking Account and Project accounts, IPFS Search Engine implementation. 

# Part - 1 Cross Linking Account and Project accounts

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

2 - User and Project id extraction from JSON-LD files;

3 - Queries Iroha 1 for User and Project accounts and checks the present values;

4 - Sets details for both User and Project accounts in Iroha 1 providing a logical link between them for later references;

5 - Queries the User and Project accounts again and checks the proper setting of details.

## Sequence Diagram

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant self as "self"
    participant Blockchain as "Iroha 1 Blockchain"
    

    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, Blockchain: Extract user and project IDs from JSONs
    Platform->>self: User ID Extraction
    Platform->>self: Project ID Extraction

    Note over Platform, Blockchain: Queries the blockchain for <br/>User and Project accounts details
    Platform->>Blockchain: Get User Account Details
    Blockchain-->>Platform: Query Response
    Platform->>Blockchain: Get Project Account Details
    Blockchain-->>Platform: Query Response
    
    Note over Platform, Blockchain: Set details for User and Project accounts
    Platform->>Blockchain: Set User Details in Blockchain
    Blockchain-->>Platform: User Details Set Successfully
    Platform->>Blockchain: Set Project Details in Blockchain
    Blockchain-->>Platform: Project Details Set Successfully
    
    Note over Platform, Blockchain: Queries the blockchain to <br/>confirm proper setting of details
    Platform->>Blockchain: Get User Account Details
    Blockchain-->>Platform: Query Response
    Platform->>Blockchain: Get Project Account Details
    Blockchain-->>Platform: Query Response
    
```

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [1]:
from Crypto.Hash import keccak
import os
import binascii
from iroha import IrohaCrypto
from iroha import Iroha, IrohaGrpc
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import sys
# import csv
import json
import icecream as ic

if sys.version_info[0] < 3:
    raise Exception("Python 3 or a more recent version is required.")

# Load configuration from config.json file
config_path = "config.json"  # Update this path as needed
with open(config_path, "r") as f:
    config = json.load(f)

IROHA_HOST_ADDR = config["IROHA_HOST_ADDR"]
IROHA_PORT = config["IROHA_PORT"]
ADMIN_ACCOUNT_ID = config["ADMIN_ACCOUNT_ID"]
ADMIN_PRIVATE_KEY = config["ADMIN_PRIVATE_KEY"]

iroha = Iroha(ADMIN_ACCOUNT_ID)
net = IrohaGrpc("{}:{}".format(IROHA_HOST_ADDR, IROHA_PORT))


@integration_helpers.trace
def create_contract():
    bytecode = "608060405234801561001057600080fd5b5073a6abc17819738299b3b2c1ce46d55c74f04e290c6000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610b4c806100746000396000f3fe608060405234801561001057600080fd5b506004361061004c5760003560e01c80635bdb3a41146100515780637949a1b31461006f578063b7d66df71461009f578063d4e804ab146100cf575b600080fd5b6100596100ed565b6040516100669190610879565b60405180910390f35b61008960048036038101906100849190610627565b61024c565b6040516100969190610879565b60405180910390f35b6100b960048036038101906100b49190610693565b6103bb565b6040516100c69190610879565b60405180910390f35b6100d761059b565b6040516100e4919061085e565b60405180910390f35b606060006040516024016040516020818303038152906040527f5bdb3a41000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16836040516101be9190610830565b600060405180830381855af49150503d80600081146101f9576040519150601f19603f3d011682016040523d82523d6000602084013e6101fe565b606091505b509150915081610243576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161023a9061091e565b60405180910390fd5b80935050505090565b60606000838360405160240161026392919061089b565b6040516020818303038152906040527f7949a1b3000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168360405161032a9190610830565b600060405180830381855af49150503d8060008114610365576040519150601f19603f3d011682016040523d82523d6000602084013e61036a565b606091505b5091509150816103af576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016103a69061091e565b60405180910390fd5b80935050505092915050565b606060008484846040516024016103d4939291906108d2565b6040516020818303038152906040527fb7d66df7000000000000000000000000000000000000000000000000000000007bffffffffffffffffffffffffffffffffffffffffffffffffffffffff19166020820180517bffffffffffffffffffffffffffffffffffffffffffffffffffffffff8381831617835250505050905060008060008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168360405161049b9190610830565b600060405180830381855af49150503d80600081146104d6576040519150601f19603f3d011682016040523d82523d6000602084013e6104db565b606091505b509150915081610520576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016105179061091e565b60405180910390fd5b8460405161052e9190610847565b6040518091039020866040516105449190610847565b60405180910390208860405161055a9190610847565b60405180910390207f5e1b38cd47cf21b75d5051af29fa321eedd94877db5ac62067a076770eddc9d060405160405180910390a48093505050509392505050565b60008054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b60006105d26105cd84610963565b61093e565b9050828152602081018484840111156105ea57600080fd5b6105f5848285610a14565b509392505050565b600082601f83011261060e57600080fd5b813561061e8482602086016105bf565b91505092915050565b6000806040838503121561063a57600080fd5b600083013567ffffffffffffffff81111561065457600080fd5b610660858286016105fd565b925050602083013567ffffffffffffffff81111561067d57600080fd5b610689858286016105fd565b9150509250929050565b6000806000606084860312156106a857600080fd5b600084013567ffffffffffffffff8111156106c257600080fd5b6106ce868287016105fd565b935050602084013567ffffffffffffffff8111156106eb57600080fd5b6106f7868287016105fd565b925050604084013567ffffffffffffffff81111561071457600080fd5b610720868287016105fd565b9150509250925092565b610733816109e2565b82525050565b600061074482610994565b61074e81856109aa565b935061075e818560208601610a23565b61076781610ab6565b840191505092915050565b600061077d82610994565b61078781856109bb565b9350610797818560208601610a23565b80840191505092915050565b60006107ae8261099f565b6107b881856109c6565b93506107c8818560208601610a23565b6107d181610ab6565b840191505092915050565b60006107e78261099f565b6107f181856109d7565b9350610801818560208601610a23565b80840191505092915050565b600061081a6027836109c6565b915061082582610ac7565b604082019050919050565b600061083c8284610772565b915081905092915050565b600061085382846107dc565b915081905092915050565b6000602082019050610873600083018461072a565b92915050565b600060208201905081810360008301526108938184610739565b905092915050565b600060408201905081810360008301526108b581856107a3565b905081810360208301526108c981846107a3565b90509392505050565b600060608201905081810360008301526108ec81866107a3565b9050818103602083015261090081856107a3565b9050818103604083015261091481846107a3565b9050949350505050565b600060208201905081810360008301526109378161080d565b9050919050565b6000610948610959565b90506109548282610a56565b919050565b6000604051905090565b600067ffffffffffffffff82111561097e5761097d610a87565b5b61098782610ab6565b9050602081019050919050565b600081519050919050565b600081519050919050565b600082825260208201905092915050565b600081905092915050565b600082825260208201905092915050565b600081905092915050565b60006109ed826109f4565b9050919050565b600073ffffffffffffffffffffffffffffffffffffffff82169050919050565b82818337600083830152505050565b60005b83811015610a41578082015181840152602081019050610a26565b83811115610a50576000848401525b50505050565b610a5f82610ab6565b810181811067ffffffffffffffff82111715610a7e57610a7d610a87565b5b80604052505050565b7f4e487b7100000000000000000000000000000000000000000000000000000000600052604160045260246000fd5b6000601f19601f8301169050919050565b7f4572726f722063616c6c696e67207365727669636520636f6e7472616374206660008201527f756e6374696f6e0000000000000000000000000000000000000000000000000060208201525056fea26469706673582212206ad40afbd4cc9c87ae154542d003c9538e4b89473a13cadd3cbf618ea181206864736f6c63430008040033"
    """Bytecode was generated using remix editor  https://remix.ethereum.org/ from file detail.sol. """
    tx = iroha.transaction(
        [iroha.command("CallEngine", caller=ADMIN_ACCOUNT_ID, input=bytecode)]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    net.send_tx(tx)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    for status in net.tx_status_stream(tx):
        print(status)
    return hex_hash

hash = create_contract()
integration_helpers.get_engine_receipts_result(hash)
print("done")

	Entering "create_contract"
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
	Leaving "create_contract"
	Entering "get_engine_receipts_result"

	Leaving "get_engine_receipts_result"
done


2 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` with a entry number related to an existing object in `datasets/accounts.json` and `datasets/projects.json`

In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 0

4 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [3]:
import json
import os
import binascii

address = integration_helpers.get_engine_receipts_address(hash)


# Function to link details using blockchain
def set_account_detail(address, account, key, value):
    params = integration_helpers.get_first_four_bytes_of_keccak(
        b"setAccountDetail(string,string,string)"
    )
    no_of_param = 3
    for x in range(no_of_param):
        params = params + integration_helpers.left_padded_address_of_param(
            x, no_of_param
        )
    params = params + integration_helpers.argument_encoding(account)  # account id
    params = params + integration_helpers.argument_encoding(key)  # key
    params = params + integration_helpers.argument_encoding(value)  # value
    tx = iroha.transaction(
        [
            iroha.command(
                "CallEngine", caller=ADMIN_ACCOUNT_ID, callee=address, input=params
            )
        ]
    )
    IrohaCrypto.sign_transaction(tx, ADMIN_PRIVATE_KEY)
    response = net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    hex_hash = binascii.hexlify(IrohaCrypto.hash(tx))
    return hex_hash

# Function to update user account with linked project
def update_user_account_link(user_account_id, linked_project_id, accounts_filename="datasets/accounts.json"):
    try:
        if os.path.exists(accounts_filename):
            with open(accounts_filename, mode='r', encoding='utf-8') as file:
                data = json.load(file)
        else:
            print(f"{accounts_filename} does not exist.")
            return

        user_found = False

        # Look for the user account and update it
        for entry in data["@graph"]:
            if entry["@type"] == "foaf:Person" and entry.get("foaf:holdsAccount", {}).get("schema:identifier") == user_account_id:
                entry["schema:linked_project"] = linked_project_id
                user_found = True
                print(f"Updated user account {user_account_id} with linked project {linked_project_id}")
                break

        if not user_found:
            print(f"User account {user_account_id} not found.")

        # Write back the updated data
        with open(accounts_filename, mode='w', encoding='utf-8') as file:
            json.dump(data, file, indent=4)

    except Exception as e:
        print(f"Error updating user account in JSON-LD: {str(e)}")


# Function to update project account with linked user
def update_project_account_link(project_account_id, linked_user_id, projects_filename="datasets/projects.json"):
    try:
        if os.path.exists(projects_filename):
            with open(projects_filename, mode='r', encoding='utf-8') as file:
                data = json.load(file)
        else:
            print(f"{projects_filename} does not exist.")
            return

        project_found = False

        # Look for the project account and update it
        for entry in data["@graph"]:
            if entry["@type"] == "schema:ResearchProject" and entry.get("schema:identifier") == project_account_id:
                entry["schema:linked_user"] = linked_user_id
                project_found = True
                print(f"Updated project account {project_account_id} with linked user {linked_user_id}")
                break

        if not project_found:
            print(f"Project account {project_account_id} not found.")

        # Write back the updated data
        with open(projects_filename, mode='w', encoding='utf-8') as file:
            json.dump(data, file, indent=4)

    except Exception as e:
        print(f"Error updating project account in JSON-LD: {str(e)}")


# Function to read accounts from JSON-LD
def read_user_accounts_from_jsonld(file_path):
    with open(file_path, mode='r', encoding='utf-8') as file:
        data = json.load(file)
        user_accounts = []
        for entry in data["@graph"]:
            if entry["@type"] == "foaf:Person":
                account_id = entry.get("foaf:holdsAccount", {}).get("schema:identifier")
                if account_id:
                    user_accounts.append({'account_id': account_id})
        return user_accounts


def read_project_accounts_from_jsonld(file_path):
    with open(file_path, mode='r', encoding='utf-8') as file:
        data = json.load(file)
        project_accounts = []
        for entry in data["@graph"]:
            if entry["@type"] == "schema:ResearchProject":
                project_id = entry.get("schema:identifier")
                if project_id:
                    project_accounts.append({'account_id': project_id})
        return project_accounts


# Example execution of the previous snippet
address = integration_helpers.get_engine_receipts_address(hash)

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
print(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
print(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")


	Entering "get_engine_receipts_address"
	Leaving "get_engine_receipts_address"
	Entering "get_engine_receipts_address"
	Leaving "get_engine_receipts_address"
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
('STATELESS_VALIDATION_SUCCESS', 1, 0)
('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
Updated user account peaceful_newton@test with linked project 09938@test
Updated project account 09938@test with linked user peaceful_newton@test
User account peaceful_newton@test linked to project 09938@test
Project account 09938@test linked to user peaceful_newton@test


3 - Queries Iroha 1 for User account and checks its values

In [4]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=user_account['account_id'])
# print(query)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# print(response)

user_data = response.account_detail_response
user_details = user_data.detail


print(f'User Account id = {user_account}, {user_details}')



User Account id = {'account_id': 'peaceful_newton@test'}, { "admin@test" : { "linked_project" : "09938@test", "user_json_ld_cid" : "QmVmA19rWkcCW2Y7phKrBbPUCWBZhkkgmskz2NXA3jLPbq" } }


3 - Queries Iroha 1 for Project account and checks its values

In [5]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=project_account['account_id'])
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# print(response)
project_data = response.account_detail_response
project_details = project_data.detail
print(f'Project Account id = {project_account}, {project_details}')

Project Account id = {'account_id': '09938@test'}, { "admin@test" : { "file_1_CID" : "QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4", "file_1_metadata_CID" : "QmZXzHYzx3W4pvxTNeG5esoVvpEzqARMj7mgbv2RfqH9jT", "linked_user" : "peaceful_newton@test", "project_metadata_cid" : "QmRA1y8QYdUp6VMHkLnWtfqgGgxRUFZaRpBYTgSGSoLVjQ" } }


# Part2 - Querying Project Metadata 

## Activities

6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.


## Sequence Diagram

```mermaid
%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"
    participant IPFS as "Interplanetary File System"
    participant FrontEnd as "Front End"

    Note over Platform, Blockchain: Queries the user account <br/> and get the project id 
    Platform->>Blockchain: Query User Account Details
    Blockchain-->>Platform: Query Response
        
    Note over Platform, Blockchain: Queries the Project Account details <br/> and get the project metadata CID 
    Platform->>Blockchain: Query Project Account Details
    Blockchain-->>Platform: Query Response

    Note over Platform, IPFS: Process and displays the metadata CID 
    Platform->>IPFS: Sends the project metadata CID
    IPFS-->>Platform: Sends back the project metadata JSON
    Platform->>FrontEnd: Displays the project metadata JSON   
```

6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.

In [6]:
from ipfs_functions import *

print(user_details)

# Process the account details response
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object
print(user_details_dict)

# Now you can access the specific key like this
linked_project = user_details_dict["admin@test"]["linked_project"]
print(linked_project)

{ "admin@test" : { "linked_project" : "09938@test", "user_json_ld_cid" : "QmVmA19rWkcCW2Y7phKrBbPUCWBZhkkgmskz2NXA3jLPbq" } }
{'admin@test': {'linked_project': '09938@test', 'user_json_ld_cid': 'QmVmA19rWkcCW2Y7phKrBbPUCWBZhkkgmskz2NXA3jLPbq'}}
09938@test


In [7]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail', account_id=linked_project)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# print(response)
project_data = response.account_detail_response
project_details = project_data.detail
print(f'Project Account id = {project_account}, {project_details}')

Project Account id = {'account_id': '09938@test'}, { "admin@test" : { "file_1_CID" : "QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4", "file_1_metadata_CID" : "QmZXzHYzx3W4pvxTNeG5esoVvpEzqARMj7mgbv2RfqH9jT", "linked_user" : "peaceful_newton@test", "project_metadata_cid" : "QmRA1y8QYdUp6VMHkLnWtfqgGgxRUFZaRpBYTgSGSoLVjQ" } }


In [8]:
# Convert the JSON string to a Python dictionary
project_details_dict = json.loads(project_details)

# Now you can access the specific key like this
project_metadata_cid = project_details_dict["admin@test"]["project_metadata_cid"]
print(project_metadata_cid)


project_metadata = download_json_from_ipfs(project_metadata_cid)

# print(20*"-")

print(project_metadata)

QmRA1y8QYdUp6VMHkLnWtfqgGgxRUFZaRpBYTgSGSoLVjQ
{'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This study aims to investigate the effects of e-waste on green transportation and propose strategies for improvement.', 'schema:endDate': '2028-03-20', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'Asian Development Bank'}, 'schema:keywords': ['e-waste', 'green transportation', 'supply chain resilience'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Svalbard, Norway'}, 'schema:name': 'Exploring the Effects of e-waste on green transportation', 'schema:startDate': '2020-12-30'}


# Part 3 - File Operations 

7 -  Sends every file in the `upload` directory to IPFS, extracts theirs respective metadata with Apache Tika and sends it to IPFS, get the CIDs back and store in Iroha as details of the project account.

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Metadata Extractor
    participant Indexer
    participant Blockchain as "Iroha 1 Blockchain"
    participant IPFS as "Interplanetary File System"

    Note over Platform, IPFS: File Operations 
    Platform->>IPFS: Upload local file to IPFS
    IPFS-->>Platform: Send back file CIDs
    Platform->>Blockchain: Set CID as Project Account Details
    Blockchain-->>Platform:Details set successfully

    Note over Platform, IPFS: File Metadata Operations
    Platform->>Metadata Extractor: Parse file and extract metadata 
    Metadata Extractor->>Indexer: Send file metadata for indexing
    Indexer->>IPFS: Store file metadata JSON
    IPFS-->>Platform: Send back file metadata JSON CIDs
    Platform->>Blockchain: Set file metadata JSON CID as Project Account Details
    Blockchain-->>Platform:Details set successfully
         
```

In [11]:
### New version

from whoosh.index import create_in, open_dir, EmptyIndexError  # Import EmptyIndexError
from metadata_helper import *
import os
import logging
import mimetypes
from tika import parser  # Import Apache Tika

print(project_account['account_id'])

# Function to extract only Dublin Core related metadata
def extract_dublin_core(metadata):
    """Extracts only Dublin Core related metadata."""
    return {k: v for k, v in metadata.items() if k.startswith('dc:') or k.startswith('dcterms:')}


# Function to parse and index documents from a directory
def parse_documents_in_directory(directory_path, schema, linked_project, recreate=False):
    """Parses documents in a directory and indexes them."""
    ix = recreate_index(schema) if recreate else create_in("indexdir", schema)
    writer = get_writer_with_retry(ix)
    index = 1

    for filename in os.listdir(directory_path):
        logging.info(f"Processing file: {filename}")

        if not os.path.basename(filename).startswith('.'):
            file_path = os.path.join(directory_path, filename)

            try:
                # Upload the file to IPFS and get its CID
                file_cid = upload_file_to_ipfs(file_path)
                logging.info(f"File {filename} uploaded to IPFS with CID: {file_cid}")

                
                # Update the project details in the Iroha 1 blockchain
                hash = set_account_detail(address, project_account['account_id'], f"file_{index}_CID", file_cid)


                # Parse the document using Apache Tika

                try:
                    parsed_document = parser.from_file(file_path)
                    print(parsed_document)
                except Exception as e:
                        logging.error(f"Error parsing file with Tika: {e}")
                        continue
                
                if parsed_document:
                    metadata = parsed_document.get('metadata', {})
                    full_text = parsed_document.get("content", "").strip() or "No content extracted"

                    # Extract Dublin Core related metadata
                    dublin_core_metadata = extract_dublin_core(metadata)
                    
                    # Normalize and upload JSON metadata to IPFS
                    normalized_metadata = {
                        'project_id': linked_project,
                        'cid': upload_json_to_ipfs(metadata),  # Upload the full metadata
                        'name': filename,
                        'size': os.path.getsize(file_path),
                        'filetype': mimetypes.guess_type(filename)[0] or "unknown",
                        'title': normalize_metadata_value(metadata.get("dc:title", f"Document {index}")),
                        'creator': normalize_metadata_value(metadata.get("dc:creator", "Unknown")),
                        'language': normalize_metadata_value(metadata.get("dc:language", "en")),
                        'subject': normalize_metadata_value(metadata.get("dc:subject", "")),
                        'description': normalize_metadata_value(metadata.get("dc:description", "")),
                        'publisher': normalize_metadata_value(metadata.get("dc:publisher", "Unknown")),
                        'date': normalize_metadata_value(metadata.get("dc:date", "")),
                        'abstract': normalize_metadata_value(metadata.get("dc:abstract", "")),
                        'format': normalize_metadata_value(metadata.get("dc:format", "")),
                        'created': normalize_metadata_value(metadata.get("dcterms:created", "")),
                        'modified': normalize_metadata_value(metadata.get("dcterms:modified", ""))
                    }
                    
                    logging.info(f"Indexed {filename} with CID: {normalized_metadata['cid']}")

                    # Add document to the Whoosh index
                    add_document(writer, normalized_metadata, full_text)

                    # Print extracted Dublin Core metadata
                    print("Dublin Core Metadata:")
                    print(json.dumps(dublin_core_metadata, indent=4))

                    # Update project entry with file data and Dublin Core metadata
                    update_project_entry_with_file_data(
                        linked_project, file_cid, normalized_metadata['cid'], dublin_core_metadata
                    )
                    logging.info("updated project entry")
                    
                    # Upload the metadata to IPFS and get its CID
                    print(40*"-")
                    metadata_cid = upload_json_to_ipfs(metadata)
                    logging.info(f"File {filename} uploaded to IPFS with CID: {metadata_cid}")
                                        
                    #Sets the project account detail with the file metadata
                    print(40*"-")

                    # print(project_account['account_id'])
                    # print(index)
                    hash = set_account_detail(
                        address, project_account['account_id'], f"file_{index}_metadata_CID", metadata_cid
                        )

                    print(40*"-")
                    
                    #-
                    
                else:
                    logging.error(f"Parsing failed for '{filename}'.")

            except Exception as e:
                logging.error(f"Error processing file '{filename}': {e}")
                continue

        logging.info("-" * 40)
        index += 1

    writer.commit()  # Commit changes once all files are processed
    logging.info("All documents processed and index committed.")


# Function to set up the Whoosh index directory
def setup_index(schema):
    """Sets up the Whoosh index directory and returns the index object."""
    index_dir = "indexdir"
    if not os.path.exists(index_dir):
        os.mkdir(index_dir)
        logging.info("Index directory created.")
        ix = create_in(index_dir, schema)
    else:
        try:
            ix = open_dir(index_dir)
            logging.info("Opened existing index.")
        except EmptyIndexError:
            logging.warning("Index is empty. Creating a new index.")
            ix = create_in(index_dir, schema)
    return ix

# ic(linked_project)

# Define the schema (including Dublin Core fields)
schema = Schema(
    project_id=TEXT(stored=True),
    cid=ID(stored=True),
    name=TEXT(stored=True),
    size=NUMERIC(stored=True),
    filetype=TEXT(stored=True),
    title=TEXT(stored=True),
    creator=TEXT(stored=True),
    language=TEXT(stored=True),
    subject=TEXT(stored=True),
    description=TEXT(stored=True),
    publisher=TEXT(stored=True),
    date=TEXT(stored=True),
    abstract=TEXT(stored=True),
    format=TEXT(stored=True),
    created=TEXT(stored=True),  # Store as string (e.g., ISO format 'YYYY-MM-DD')
    modified=TEXT(stored=True),  # Store as string (e.g., ISO format 'YYYY-MM-DD')
    full_text=TEXT(stored=False)
)

# Setup index directory
ix = setup_index(schema)

# Example document parsing and indexing execution
directory_path = "upload"

linked_project = project_account['account_id']  # Example placeholder, adjust as needed

# Parse documents in the specified directory
parse_documents_in_directory(directory_path, schema, linked_project, recreate=False)



INFO:root:Opened existing index.
INFO:root:Processing file: Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf


09938@test


INFO:root:File Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf uploaded to IPFS with CID: QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4


('STATELESS_VALIDATION_SUCCESS', 1, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)


INFO:root:Indexed Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf with CID: QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj
INFO:root:Document Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf indexed successfully.
INFO:root:updated project entry
INFO:root:File Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf uploaded to IPFS with CID: QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj


{'metadata': {'pdf:PDFVersion': '1.7', 'pdf:docinfo:title': 'Aeroacoustic airfoil shape optimization enhanced by autoencoders', 'xmp:CreatorTool': 'Elsevier', 'pdf:hasXFA': 'false', 'access_permission:can_print_degraded': 'true', 'X-TIKA:Parsed-By-Full-Set': ['org.apache.tika.parser.DefaultParser', 'org.apache.tika.parser.pdf.PDFParser'], 'X-TIKA:content_handler': 'ToTextContentHandler', 'pdf:num3DAnnotations': '0', 'dc:format': 'application/pdf; version=1.7', 'pdf:docinfo:custom:robots': 'noindex', 'pdf:docinfo:creator_tool': 'Elsevier', 'access_permission:fill_in_form': 'true', 'pdf:docinfo:custom:CreationDate--Text': '7th February 2023', 'pdf:hasCollection': 'false', 'pdf:encrypted': 'false', 'dc:title': 'Aeroacoustic airfoil shape optimization enhanced by autoencoders', 'pdf:containsNonEmbeddedFont': 'false', 'pdf:docinfo:custom:CrossMarkDomains[1]': 'elsevier.com', 'robots': 'noindex', 'pdf:docinfo:subject': 'Expert Systems With Applications, 217 (2023) 119513. doi:10.1016/j.eswa.

INFO:root:----------------------------------------


('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)
----------------------------------------


INFO:root:All documents processed and index committed.


In [12]:
# Example search usage
keyword = "must"
search_index(keyword, ix)


INFO:root:CID: QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj, Project: 09938@test, Name: aeroacoustic-airfoil-shape-optimization-enhance_2023_expert-systems-with-app.pdf, Title: aeroacoustic airfoil shape optimization enhanced by autoencoders, Creator: jiaqing kou, Size: 1324070 bytes


(['09938@test'],
 [{'cid': 'QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj',
   'project_id': '09938@test',
   'name': 'aeroacoustic-airfoil-shape-optimization-enhance_2023_expert-systems-with-app.pdf',
   'title': 'aeroacoustic airfoil shape optimization enhanced by autoencoders',
   'creator': 'jiaqing kou',
   'size': 1324070}])

8 -  Search Engine

In [13]:
project_ids = search_index(keyword, ix)
print(project_ids)  # prints the list of project ids found in thvc e results

INFO:root:CID: QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj, Project: 09938@test, Name: aeroacoustic-airfoil-shape-optimization-enhance_2023_expert-systems-with-app.pdf, Title: aeroacoustic airfoil shape optimization enhanced by autoencoders, Creator: jiaqing kou, Size: 1324070 bytes


(['09938@test'], [{'cid': 'QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj', 'project_id': '09938@test', 'name': 'aeroacoustic-airfoil-shape-optimization-enhance_2023_expert-systems-with-app.pdf', 'title': 'aeroacoustic airfoil shape optimization enhanced by autoencoders', 'creator': 'jiaqing kou', 'size': 1324070}])


In [14]:
# keyword = "example_keyword"

# Initialize objects and variables


# project_ids = search_index(keyword, ix)
if project_ids:
    get_project_details(project_ids, net, iroha)

TypeError: unhashable type: 'list'

In [15]:
# Example usage (replace `ix`, `net`, `iroha`, and `keyword` with actual values)
check_cid_in_project_details(keyword, ix, net, iroha)


INFO:root:CID: QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj, Project: 09938@test, Name: aeroacoustic-airfoil-shape-optimization-enhance_2023_expert-systems-with-app.pdf, Title: aeroacoustic airfoil shape optimization enhanced by autoencoders, Creator: jiaqing kou, Size: 1324070 bytes


TypeError: unhashable type: 'list'

In [19]:
from itertools import chain

def main(keyword, ix, net, iroha):
    # Step 1: Search index with the provided keyword
    project_ids, search_results = search_index(keyword, ix)

    # Check if both project_ids and search_results are correctly received
    if not project_ids:
        logging.info("No project IDs found.")
    if not search_results:
        logging.info("No search results found.")

    # Flatten project_ids if it contains nested lists
    if any(isinstance(i, list) for i in project_ids):
        project_ids = list(chain.from_iterable(project_ids))

    # Step 2: Fetch project details for matched project IDs
    project_details = get_project_details(project_ids, net, iroha)
    
    # Step 3: Generate the knowledge graph with CID matching
    generate_knowledge_graph(search_results, project_details)

# Example usage
main("autoencoders", ix, net, iroha)


INFO:root:CID: QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj, Project: 09938@test, Name: aeroacoustic-airfoil-shape-optimization-enhance_2023_expert-systems-with-app.pdf, Title: aeroacoustic airfoil shape optimization enhanced by autoencoders, Creator: jiaqing kou, Size: 1324070 bytes
INFO:root:Project Account id = 09938@test, { "admin@test" : { "file_1_CID" : "QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4", "file_1_metadata_CID" : "QmVvDdQCkCcDpPJb2Sa6VDtgwVYAWGeWqCP1biDkRgi7kj", "linked_user" : "peaceful_newton@test", "project_metadata_cid" : "QmRA1y8QYdUp6VMHkLnWtfqgGgxRUFZaRpBYTgSGSoLVjQ" } }


AttributeError: 'str' object has no attribute 'items'

9 - Query the project account to verify the details update

In [ ]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail', account_id=linked_project)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# print(response)
project_data = response.account_detail_response
project_details = project_data.detail
print(f'Project Account id = {project_account}, {project_details}')

Project Account id = {'account_id': '12694@test'}, { "admin@test" : { "file_1_CID" : "QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4", "file_1_metadata_CID" : "QmaBLUUdGotvWFhgEQqVKcbZk2A6KZWvQ8a1tcV89tXD65", "linked_user" : "youthful_curie@test", "project_metadata_cid" : "QmXSV2zghFun8rT44hwaWwNGFGMBnM1jdQVbghFunDSumL" } }


10 - Read CIDs from Iroha and download file metadata and files from IPFS to the project home directory

```mermaid
%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant self as "self"
    participant Blockchain as "Iroha 1 Blockchain"
    participant IPFS as "Interplanetary File System"
    participant FrontEnd as "Front End"
    
       
    Note over Platform, Blockchain: Queries the Project Account details and get details
    Platform->>Blockchain: Query Project Account Details
    Blockchain->>Platform: Query Response

    Note over Platform, IPFS: Process project account metadata
    Platform->>self: Parse Project Details JSON and retrieve file CIDs

    Note over Platform, IPFS: Download file from IPFS 
    Platform->>IPFS: Sends the file CID
    IPFS->>Platform: Sends back the file
    Platform->>FrontEnd:    Saves the file locally and display info and status

11 - Read details from the project account retrieve the CID of every file, download the it file from IPFS and store locally.

In [ ]:
import os
import json
from clean_file_name import clean_file_name as clean_name
from ipfs_functions import download_file_from_ipfs, download_json_from_ipfs

# Convert the JSON string to a Python dictionary
project_details_dict = json.loads(project_details)
project_account_id = project_account['account_id']

print(f"Processing project account: {project_account_id}")

# Get the value of the dictionary (actual file metadata)
files_metadata = project_details_dict['admin@test']
# print(files_metadata)

# Iterate through files metadata, skip unneeded keys, and process files and metadata CIDs
for key, value in files_metadata.items():
    if key in ['linked_user', 'project_metadata_cid']:
        print(f"Skipping key: {key}")
        continue

    # Distinguish between file CID and metadata CID
    if 'metadata_CID' not in key:
        file_CID = value
        print(f"Processing file CID: {file_CID} for key: {key}")
    else:
        file_metadata_key = '_'.join(key.split('_')[:-2])
        file_metadata_CID = value
        print(f"Processing metadata CID: {file_metadata_CID} for key: {file_metadata_key}")

        # Download and process metadata JSON
        file_metadata_json = download_json_from_ipfs(file_metadata_CID)
        # print(file_metadata_json)

        # Ensure 'resourceName' exists in metadata and process the file download
        if 'resourceName' in file_metadata_json:
            raw_file_name = file_metadata_json['resourceName']
            cleaned_file_name = clean_name(raw_file_name)  # Renamed to avoid conflict
            print(f"Cleaned file name: {cleaned_file_name}")

            # Create user-specific download directory if it doesn't exist
            download_directory = os.path.join("download", project_account_id)
            os.makedirs(download_directory, exist_ok=True)
            print(f"Download directory ready: {download_directory}")

            # Download file using the file CID
            file_path = os.path.join(download_directory, cleaned_file_name)
            print(f"Downloading file to: {file_path}")
            download_file_from_ipfs(file_CID, file_path)
            print("-" * 40)
        else:
            print(f"No 'resourceName' found for metadata CID: {file_metadata_CID}")


Processing project account: 12694@test
Processing file CID: QmXh37WXcrLXkJX2cAPjbPdKZ2cxJXD58XX6eU1Zc41ka4 for key: file_1_CID
Processing metadata CID: QmaBLUUdGotvWFhgEQqVKcbZk2A6KZWvQ8a1tcV89tXD65 for key: file_1
Cleaned file name: Aeroacoustic-airfoil-shape-optimization-enhance_2023_Expert-Systems-with-App.pdf
Download directory ready: download/12694@test
----------------------------------------
Skipping key: linked_user
Skipping key: project_metadata_cid
